
#**Quantitative Momentum Strategy**
"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

Library Imports **bold text**
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.



In [4]:
!pip install xlsxwriter

     |████████████████████████████████| 153kB 5.1MB/s 


In [5]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

In [6]:
stocks = pd.read_csv('/content/sp_500_stocks.csv')
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [18]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avg10Volume': 85313809,
 'avg30Volume': 93607863,
 'beta': 1.458518082973437,
 'companyName': 'Apple Inc',
 'day200MovingAvg': 127.47,
 'day30ChangePercent': -0.05934094375884424,
 'day50MovingAvg': 130.64,
 'day5ChangePercent': -0.016578279280138022,
 'dividendYield': 0.006642626574485906,
 'employees': 152108,
 'exDividendDate': '2021-05-04',
 'float': 0,
 'marketcap': 2171976897322,
 'maxChangePercent': 49.158609411865136,
 'month1ChangePercent': -0.05947871828470459,
 'month3ChangePercent': -0.0337079593508661,
 'month6ChangePercent': 0.07364973589473772,
 'nextDividendDate': '',
 'nextEarningsDate': '2021-07-27',
 'peRatio': 27.782782708942847,
 'sharesOutstanding': 17494603708,
 'ttmDividendRate': 0.840286106511383,
 'ttmEPS': 4.65,
 'week52change': 0.591493969914573,
 'week52high': 144.86,
 'week52highSplitAdjustOnly': 149.02,
 'week52low': 80.81,
 'week52lowSplitAdjustOnly': 80.65,
 'year1ChangePercent': 0.6067588711073302,
 'year2ChangePercent': 1.7800690006675934,
 'year5Ch

In [19]:
data['year1ChangePercent']

0.6067588711073302

# Executing A Batch API Call & Building Our DataFrame
Jit's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [20]:

# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [21]:

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,138.30,0.664381,N/A
1,AAL,23.13,1.32164,N/A
2,AAP,199.39,0.477726,N/A
3,AAPL,125.78,0.616587,N/A
4,ABBV,121.17,0.322776,N/A
...,...,...,...,...
500,YUM,124.49,0.402119,N/A
501,ZBH,170.92,0.39113,N/A
502,ZBRA,505.45,1.05399,N/A
503,ZION,60.40,0.981121,N/A


# Removing Low-Momentum Stocks
The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.

In [22]:

final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,FCX,41.40,3.65808,N/A
1,LB,67.58,3.56957,N/A
2,GPS,33.38,3.03624,N/A
3,IVZ,26.96,2.87923,N/A
4,MOS,36.78,2.15524,N/A
5,KSS,56.51,2.05052,N/A
6,TPR,44.14,2.01732,N/A
7,SIVB,597.10,1.98762,N/A
8,DFS,116.28,1.88444,N/A
9,PWR,95.48,1.80514,N/A


# Calculating the Number of Shares to Buy
Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [23]:

def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:5000
5000


In [25]:
position_size = float(portfolio_size) / len(final_dataframe.index)
print("position_size: ",position_size)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

position_size:  98.03921568627452


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,FCX,41.40,3.65808,2
1,LB,67.58,3.56957,1
2,GPS,33.38,3.03624,2
3,IVZ,26.96,2.87923,3
4,MOS,36.78,2.15524,2
5,KSS,56.51,2.05052,1
6,TPR,44.14,2.01732,2
7,SIVB,597.10,1.98762,0
8,DFS,116.28,1.88444,0
9,PWR,95.48,1.80514,1



# Building a Better (and More Realistic) Momentum Strategy
Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

High-quality momentum stocks show "slow and steady" outperformance over long periods of time
Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.
The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of:

1-month price returns
3-month price returns
6-month price returns
1-year price returns
Let's start by building our DataFrame. You'll notice that I use the abbreviation hqm often. It stands for high-quality momentum.

In [26]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

In [48]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0

for row in hqm_dataframe.index:
    for time_period in time_periods:
                 
      hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0      0.627723
1      0.926733
2      0.483168
3           0.6
4      0.291089
         ...   
500    0.382178
501    0.372277
502    0.857426
503    0.831683
504    0.356436
Name: One-Year Return Percentile, Length: 505, dtype: object
0      0.491089
1      0.978218
2      0.734653
3      0.207921
4      0.445545
         ...   
500    0.384158
501    0.386139
502     0.79604
503     0.90099
504    0.186139
Name: Six-Month Return Percentile, Length: 505, dtype: object
0      0.261386
1      0.817822
2      0.746535
3      0.110891
4      0.522772
         ...   
500     0.59802
501    0.338614
502    0.186139
503    0.568317
504    0.449505
Name: Three-Month Return Percentile, Length: 505, dtype: object
0      0.346535
1      0.825743
2      0.261386
3       0.10099
4      0.718812
         ...   
500    0.548515
501    0.124752
502    0.473267
503    0.873267
504    0.665347
Name: One-Month Return Percentile, Length: 505, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,138.10,N/A,0.653399,0.627723,0.205028,0.491089,0.0458042,0.261386,-0.0115579,0.346535,N/A
1,AAL,22.70,N/A,1.32946,0.926733,0.818683,0.978218,0.21099,0.817822,0.0778773,0.825743,N/A
2,AAP,203.48,N/A,0.474929,0.483168,0.369969,0.734653,0.182259,0.746535,-0.0256416,0.261386,N/A
3,AAPL,125.60,N/A,0.622203,0.6,0.0737052,0.207921,-0.0333998,0.110891,-0.0603201,0.10099,N/A
4,ABBV,117.95,N/A,0.320967,0.291089,0.183915,0.445545,0.121573,0.522772,0.0492952,0.718812,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,125.25,N/A,0.393854,0.382178,0.156368,0.384158,0.139644,0.59802,0.019578,0.548515,N/A
501,ZBH,172.72,N/A,0.38238,0.372277,0.157436,0.386139,0.0719273,0.338614,-0.0544192,0.124752,N/A
502,ZBRA,504.46,N/A,1.06023,0.857426,0.411297,0.79604,0.00695913,0.186139,0.00629721,0.473267,N/A
503,ZION,58.60,N/A,0.983105,0.831683,0.553561,0.90099,0.131726,0.568317,0.0934799,0.873267,N/A


# Calculating the HQM Score¶
We'll now calculate our HQM Score, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The HQM Score will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [49]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

**Selecting the 50 Best Momentum Stocks**
As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the HQM Score column and dropping all but the top 50 entries.

In [51]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

**Calculating the Number of Shares to Buy**
We'll use the portfolio_input function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [52]:
portfolio_input()

Enter the value of your portfolio:5000


In [54]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
print("position_size: ", position_size)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

position_size:  98.03921568627452


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,138.10,0,0.653399,0.627723,0.205028,0.491089,0.0458042,0.261386,-0.0115579,0.346535,0.431683
1,AAL,22.70,4,1.32946,0.926733,0.818683,0.978218,0.21099,0.817822,0.0778773,0.825743,0.887129
2,AAP,203.48,0,0.474929,0.483168,0.369969,0.734653,0.182259,0.746535,-0.0256416,0.261386,0.556436
3,AAPL,125.60,0,0.622203,0.6,0.0737052,0.207921,-0.0333998,0.110891,-0.0603201,0.10099,0.25495
4,ABBV,117.95,0,0.320967,0.291089,0.183915,0.445545,0.121573,0.522772,0.0492952,0.718812,0.494554
5,ABC,121.69,0,0.325412,0.29703,0.208674,0.49703,0.139823,0.60198,-0.00534979,0.380198,0.444059
6,ABMD,293.00,0,0.43495,0.427723,0.0439989,0.150495,-0.160152,0.0237624,-0.188529,0.0039604,0.151485
7,ABT,121.72,0,0.325989,0.29901,0.0665983,0.19604,-0.0445737,0.0910891,-0.0452172,0.150495,0.184158
8,ACN,283.90,0,0.498915,0.512871,0.179772,0.437624,0.121799,0.524752,-0.0253893,0.267327,0.435644
9,ADBE,500.86,0,0.289993,0.253465,0.0558456,0.172277,0.019059,0.207921,-0.048822,0.134653,0.192079


# Formatting Our Excel Output
We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [55]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [56]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [57]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [58]:
writer.save()